In [1]:
import heapq
import itertools
from dataclasses import dataclass
from typing import List


In [2]:
@dataclass
class Order:
    order_id: int
    side: str            # "buy" or "sell"
    price: float | None  # None for MARKET
    qty: int
    remaining_qty: int
    timestamp: int
    status: str = "NEW"


@dataclass
class Trade:
    price: float
    qty: int
    buyer_id: int
    seller_id: int
    timestamp: int


In [3]:
order_id_gen = itertools.count(1)
clock = itertools.count(1)


In [4]:
class OrderBook:
    def __init__(self):
        self.bids = []   # max-heap: (-price, timestamp, order_id, order)
        self.asks = []   # min-heap: (price, timestamp, order_id, order)
        self.trades: List[Trade] = []

    def add_order(self, side, price, qty):
        order = Order(
            order_id=next(order_id_gen),
            side=side,
            price=price,
            qty=qty,
            remaining_qty=qty,
            timestamp=next(clock)
        )

        self.match(order)

        if order.remaining_qty > 0 and price is not None:
            self._add_to_book(order)

    def _add_to_book(self, order: Order):
        if order.side == "buy":
            heapq.heappush(
                self.bids,
                (-order.price, order.timestamp, order.order_id, order)
            )
        else:
            heapq.heappush(
                self.asks,
                (order.price, order.timestamp, order.order_id, order)
            )


    def match(self, incoming: Order):
        while incoming.remaining_qty > 0:
            if incoming.side == "buy":
                if not self.asks:
                    break

                best_price, _, _, resting = self.asks[0]

                if incoming.price is not None and incoming.price < best_price:
                    break

            else:  # incoming sell
                if not self.bids:
                    break

                best_price, _, _, resting = self.bids[0]
                best_price = -best_price

                if incoming.price is not None and incoming.price > best_price:
                    break

            executed_qty = min(incoming.remaining_qty, resting.remaining_qty)
            trade_price = resting.price

            buyer_id = incoming.order_id if incoming.side == "buy" else resting.order_id
            seller_id = resting.order_id if incoming.side == "buy" else incoming.order_id

            self.trades.append(
                Trade(
                    price=trade_price,
                    qty=executed_qty,
                    buyer_id=buyer_id,
                    seller_id=seller_id,
                    timestamp=next(clock)
                )
            )

            incoming.remaining_qty -= executed_qty
            resting.remaining_qty -= executed_qty

            if resting.remaining_qty == 0:
                resting.status = "FILLED"
                if resting.side == "buy":
                    heapq.heappop(self.bids)
                else:
                    heapq.heappop(self.asks)
            else:
                resting.status = "PARTIALLY_FILLED"

        incoming.status = (
            "FILLED" if incoming.remaining_qty == 0 else "PARTIALLY_FILLED"
        )


In [5]:
    def print_book(book: OrderBook):
        print("\nASKS:")
        for p, _, _, o in sorted(book.asks):
            print(f"Price {p}, Qty {o.remaining_qty}")

        print("\nBIDS:")
        for p, _, _, o in sorted(book.bids):
            print(f"Price {-p}, Qty {o.remaining_qty}")

In [6]:
book = OrderBook()

# Existing BID (must remain untouched)
book.add_order("buy", 99, 50)

# Submit ASK ladder
book.add_order("sell", 101, 10)
book.add_order("sell", 102, 20)
book.add_order("sell", 103, 30)

# Massive MARKET BUY
book.add_order("buy", None, 60)


In [7]:
print("TRADES:")
for t in book.trades:
    print(f"Price={t.price}, Qty={t.qty}")

print_book(book)


TRADES:
Price=101, Qty=10
Price=102, Qty=20
Price=103, Qty=30

ASKS:

BIDS:
Price 99, Qty 50
